<a href="https://colab.research.google.com/github/m2i101/OrAIza/blob/master/Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Deep Learning  model using Tensorflow with Keras**

We designed a model to classify the diseases in the paddy crop leaves by using Convolutional Neural Network.

## Mount the Google drive 
The image dataset is uploaded to the google drive and inorder to use it the drive is mounted.

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/rice-diseases-image-dataset

[Errno 2] No such file or directory: '/content/drive/My Drive/rice-diseases-image-dataset'
/content


## Importing the libraries required

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3 
from tensorflow.keras.layers import GlobalAveragePooling2D,Dropout,Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from random import shuffle
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
import cv2
from keras.models import load_model
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model


## Build the model
Loading the pretrained Inception V3 model

In [ ]:
#Load the Inception model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
87916544/87910968 [==============================] - 3s 0us/step


### Transfer learning with a pretrained ConvNet
Fine-tuning the top layers of pretrained network for a better accuracy

In [ ]:
CLASSES = 4
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  311


In [ ]:
fine_tune_at = 200

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

### Specify Loss Function and Optimizer

In [ ]:
model.compile(optimizer=RMSprop(lr=0.0001/10),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

## Prepare  the training and validation dataset



In [ ]:
train_dir = './RiceDiseaseDataset/train/'
validation_dir = './RiceDiseaseDataset/validation/'

### Data Preprocessing

Let's set up data generators that will read pictures in our source folders and feed them (with their labels) to our network. 

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 64
val_batchsize = 32
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(299, 299),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(299, 299),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=True)

Found 4584 images belonging to 4 classes.
Found 493 images belonging to 4 classes.


In [ ]:
print(train_generator.class_indices)

{'BrownSpot': 0, 'Healthy': 1, 'Hispa': 2, 'LeafBlast': 3}


## Train the model
Training the model for 30 epochs

In [ ]:

history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples//validation_generator.batch_size,
      verbose=1
      )


### Save the model 

In [ ]:
model.save('inception.h5')

## Check Performance

In [ ]:
model.evaluate(validation_generator,verbose=1)

16/16 [==============================] - 402s 25s/step


[0.8903744619709967, 0.8052738339132034]

In [ ]:
def predict(model, img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.inception_v3.preprocess_input(x)
    preds = model.predict(x)
    return preds[0]
img = image.load_img('/content/drive/My Drive/rice-diseases-image-dataset/Test/Brownspot/IMG_3042.jpg', target_size=(299, 299))
preds = predict(load_model('vgg16.h5'), img)
print(np.argmax(preds))

0


## Converting the trained model to tensorflow lite model

In [ ]:
#.h5 to tflite
import tensorflow as tf

model = 'inception.h5'
converter = tf.lite.TFLiteConverter.from_keras_model_file(model)
tflite_model = converter.convert()
open("inception_final.tflite", "wb").write(tflite_model)

## Model optimization
Quantize the model to reduce the size of the model

In [ ]:
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
open("inception_final_quant.tflite", "wb").write(tflite_quant_model)

## Conclusion
The model is then quantized and 
Let's develop an Android app that uses this model.